<a href="https://colab.research.google.com/github/BI-DS/EBA-3530/blob/main/Lecture_3/model_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import arma_generate_sample as ArmaProcess
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

# Model Selection

When modeling any kind of data, not only timeseries, the *final* model is unknown. Actually, given $n$ covarites the number of possible models is given by $2^n-1$.

In this notebook, we will explore three different options to arrive to the final model:
* Using information criteria models, e.g., AIC and BIC
* Using a performance metric in an out-of-sample test
* Looking into ACF and PACF plots

Note, the first two approaches are general, i.e. they can be used for other modeling techniques and not only for timeseries. On the other hand, the last approach is only for timeseries data.

In [ ]:
# load the dataset from the course's repo
url = 'https://raw.githubusercontent.com/BI-DS/EBA-3530/main/Lecture_3/timeseries_choosing_q.csv'
y = pd.read_csv(url, delimiter=',',header=None).values
print('The size of the timeseries is {}'.format(y.shape[0]))

## 1) Visualization
As usual, let's look at some plots. Start by:
 * 1.1) Plot the time series $y$. Assume that the frequency of the data is *days*.
 * 1.2) Create a figure with two subplot and plot the autocorrelation function in the first subplot and the partial autocorrelation function in the second subplot. In both cases, plot upto 20 lags.

In [ ]:
# @title 1.1) suggested solution
fig = plt.figure(figsize=(15,4))
plt.title('Time series data')
plt.xlabel('Days')
plt.ylabel('y')
plt.plot(y)
plt.show()

In [ ]:
# @title 1.2) suggested solution
fig, axs = plt.subplots(1,2,figsize=(15,4))
plot_acf(y, lags=20, ax=axs[0])
plot_pacf(y, lags=20, ax=axs[1])
plt.show()

## 2) Model Selection
Let's consider the following five models:
* M1: $y_t = y_{t-1} + e_t$
* M2: $y_t = y_{t-1} +y_{t-2} + e_t$
* M3: $y_t = y_{t-1} +y_{t-2} +y_{t-3}+ e_t$
* M4: $y_t = y_{t-1} +y_{t-2} +y_{t-3} +y_{t-4} + e_t$
* M5: $y_t = y_{t-1} +y_{t-2} +y_{t-3} +y_{t-4} +y_{t-5} + e_t$

2.1) define each of the five models using a tuple that specifies the number of lags, integrated, and moving average component of the model. Place the 5 tuples in a list.

2.2) Write a for-loop that iterates over the list you created in the above exercise. Inside the loop, fit the corresponding AR(p) model and get the AIC and BIC values. Append them in a list. Finally, after the loop has iterated over all models, get the smallest AIC and BIC value and print out the AR(p) corresponing to these values.

In [ ]:
# @title 2.1) suggested solution

# define all 5 models in a list with tupples
# each tuple specifies the number of lags in
# the AR process
orders = [(1,0,0),(2,0,0),(3,0,0),(4,0,0),(5,0,0)]

[(1, 0, 0), (2, 0, 0), (3, 0, 0), (4, 0, 0), (5, 0, 0)]


In [ ]:
# @title 2.2) suggested solution
all_aic = []
all_bic = []

# loop through the 5 models
for i,order in enumerate(orders):
  # define and fit AR model
  model = ARIMA(y,order=order).fit()
  # get AIC and BIC from the fitted model
  all_aic.append(model.aic)
  all_bic.append(model.bic)

  print('Model with {0} lag has AIC {1:.3f} and BIC {2:.3f}'\
        .format(i+1,model.aic, model.bic))

print('The winner model accordig with AIC is {} !!!'\
      .format(orders[np.argmin(all_aic)]))
print('The winner model accordig with BIC is {} !!!'\
      .format(orders[np.argmin(all_bic)]))

Model with 1 lag has AIC 2962.559 and BIC 2977.283
Model with 2 lag has AIC 2856.605 and BIC 2876.237
Model with 3 lag has AIC 2857.858 and BIC 2882.396
Model with 4 lag has AIC 2859.268 and BIC 2888.715
Model with 5 lag has AIC 2860.057 and BIC 2894.411
The winner model accordig with AIC is (2, 0, 0) !!!
The winner model accordig with BIC is (2, 0, 0) !!!


##3) Out-of-sample performance

3.1) Choose the first 950 observations to train each of the five models in 2.1), and the last 50 observations to test the out-of-sample performance of the models Use the following performance metrics to measure model performance:
* mean square error (mse)
* root mean square error (rmse)

In [ ]:
# @title 3.1) suggested solution

# define train and test data sets
no_obs_tr = 950
y_tr = y[:no_obs_tr]
y_te = y[no_obs_tr:]
all_mse = []
all_rmse = []

# loop through the 5 diff models
for i,order in enumerate(orders):
  # define and fit AR model
  model = ARIMA(y_tr, order=order).fit()

  # Forecast
  horizon = y_te.shape[0]
  y_hat = model.get_forecast(horizon).summary_frame()['mean'].values

  #calculate mse and rmse
  mse = np.mean((y_hat - y_te)**2)
  rmse = np.mean((y_hat - y_te)**2)**0.5

  # save metrics
  all_mse.append(mse)
  all_rmse.append(rmse)
  print('Model with {0} lag has mse {1:.3f} and rmse {2:.3f}'\
        .format(i+1,mse,rmse))

print('The winner model accordig with mse is {} !!!'\
      .format(orders[np.argmin(all_mse)]))
print('The winner model accordig with rmse is {} !!!'\
      .format(orders[np.argmin(all_rmse)]))

Model with 1 lag has mse 4.386 and rmse 2.094
Model with 2 lag has mse 3.753 and rmse 1.937
Model with 3 lag has mse 3.711 and rmse 1.926
Model with 4 lag has mse 3.631 and rmse 1.905
Model with 5 lag has mse 3.748 and rmse 1.936
The winner model accordig with mse is (4, 0, 0) !!!
The winner model accordig with rmse is (4, 0, 0) !!!


**Bonus Question:** use different time periods for the training sample and create a loop that iterates through them, fitting the 5 models from 2.1). Do you obtain different results?

**Tips:** You will need two loops, something like this:

```
for n in no_obs_tr:
  #here you define y_tr and y_te using n
  for i,order in enumerate(orders):
```


## 4) ACF and PACF
This approach only applies to timeseries models, e.g. AR, MA or ARMA.

According with *Shumway, R. H., Stoffer, D. S., & Stoffer, D. S. (2000). Time series analysis and its applications (Vol. 3). New York: springer*, we can select the number of lags (p) in an autoregressive (AR) model by looking into the autocorrelation function (ACF) and the partial autocorreltation function (PACF).

As shown by the table below, an AR process shows ACFs that tails off. In other words, the ACF decreases gradually. On the other hand, the PACF for an AR process cuts off after lag p. That is, becomes zero in statistical terms.  

|     | AR(p) |
| --- | ---   |
| ACF | Tails off |
| PACF| Cuts off after lag p|


4.1) Plot (again) the ACF and PACF for the time series $y$

In [ ]:
# @title 4.1) suggested solution

# plot the first 20 lags
fig, axs = plt.subplots(1,2,figsize=(15,4))
plot_acf(y, lags=20, ax=axs[0])
plot_pacf(y, lags=20, ax=axs[1])
plt.show()

The ACF (left diagram) clearly tails off and the PACF (right diagram) becomes 0 after lag 2.

**Question:** What do these results suggest?